In [4]:
import urllib.request
import numpy as np
import random
import math

In [22]:
def getData(url):
  #get data 
  contents = urllib.request.urlopen(url).read()
  text = str(contents,'utf-8')
  # data preprocessing
  sptext = text.split('\n')

  X = []
  Y = []

  for i in range(len(sptext)-1):
    data = sptext[i].split('\t')
    # print(data)
    label = data[-1]
    # print(type(data))
    data.pop()
    # print(len(data))
    X += [data]
    Y += [label]

  X = np.asarray(X, dtype = float)
  Y = np.asarray(Y, dtype = float)
  # print(X)
  # print(Y)
  return X, Y

def transform(X, Q):
  trans_X = []
  for x in X:
    x_Q = []
    for q in range(2, Q+1):
      x_q = np.zeros(len(x))
      x_q = np.power(x, q)
      x_Q = np.concatenate([x_Q, x_q])
    # print(x_Q)
    x = np.insert(x, 0, 1)
    x_Q = np.asarray(x_Q)
    x = np.concatenate([x, x_Q])
    trans_X.append(x)
  # print(trans_X[0])
  return trans_X

if __name__ == '__main__':
  train_data = "https://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw3/hw3_train.dat"
  test_data = "https://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw3/hw3_test.dat"
  train_X ,train_Y = getData(train_data)
  test_X, test_Y = getData(test_data)
  Q = 8
  trans_train_X = transform(train_X, Q)
  # print(type(trans_train_X[0]))
  trans_test_X = transform(test_X, Q)
  # print(len(trans_test_X[0]))

  # Calculate pseudo-inverse X_cross
  trans_train_X_cross = np.linalg.pinv(trans_train_X)
  # print(trainX_cross, type(trainX_cross))

  # Calculate WLIN = X_cross ‧ Y
  WLIN = np.matmul(trans_train_X_cross, train_Y)
  print(len(WLIN))

  # Calculate Ein
  EinN = 0
  for i in range(len(trans_train_X)):
    h_xn = np.matmul(WLIN.T, trans_train_X[i])
    if np.sign(h_xn) != train_Y[i]:
      EinN += 1
  # print("EinN: ", EinN)
  Ein = EinN/len(trans_train_X)
  print("Ein: ", Ein)
  
  # Calculate Eout
  EoutN = 0
  for i in range(len(trans_test_X)):
    h_xn = np.matmul(WLIN.T, trans_test_X[i])
    if np.sign(h_xn) != test_Y[i]:
      EoutN += 1
  # print("EoutN: ", EoutN)
  Eout = EoutN/len(trans_test_X)
  print("Eout: ", Eout)

  result = abs(Ein - Eout)
  print(result)

81
Ein:  0.052
Eout:  0.5096666666666667
0.4576666666666667
